In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"

import numpy as np
import sklearn

from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.datasets import make_regression
from sklearn.decomposition import SparseCoder, sparse_encode
from sklearn.linear_model import orthogonal_mp_gram

import scipy.linalg as LA
from scipy.sparse.linalg import lsqr, lsmr

# Numpy lstsq

In [2]:
def OMP(Y, T_0, D, rng=42, debug=False):
    # loss = np.empty(num_iter)
    # rng = np.random.default_rng(rng)

    X = np.zeros((Y.shape[0], D.shape[0]))

    # gram = D @ D.T
    # cov = D @ Y.T
    # X = sparse_encode(Y, 
    #                   D, 
    #                   algorithm='omp', 
    #                   n_nonzero_coefs = T_0,
    #                   gram = gram,
    #                   cov = cov,
    #                   copy_cov=False)

    for i, y in enumerate(Y):
        I = []
        D_I = np.zeros((T_0, D.shape[1]))
        r = y
        gamma = 0

        for j in range(T_0):
            D_r = np.abs(D @ r)
            k = np.argmax(D_r)
            
            I.append(k)
            D_I[j] = D[k]
            
            gamma, res, rank, s = np.linalg.lstsq(D_I[:j+1].T, y)
            if debug:
                print(gamma.shape)
                print(D_I[:j+1].T.shape)
                print(y.shape)
            r = y - D_I[:(j+1)].T @ gamma 
            if debug:
                print(np.sum(r * r))
                print(res)

        X[i, I] = gamma
        
    return X

# Numpy solve

In [3]:
def OMP_2(Y, T_0, D, rng=42, debug=False):
    # loss = np.empty(num_iter)
    # rng = np.random.default_rng(rng)

    X = np.zeros((Y.shape[0], D.shape[0]))

    # gram = D @ D.T
    # cov = D @ Y.T
    # X = sparse_encode(Y, 
    #                   D, 
    #                   algorithm='omp', 
    #                   n_nonzero_coefs = T_0,
    #                   gram = gram,
    #                   cov = cov,
    #                   copy_cov=False)

    for i, y in enumerate(Y):
        I = []
        D_I = np.zeros((T_0, D.shape[1]))
        r = y
        gamma = 0

        for j in range(T_0):
            D_r = np.abs(D @ r)
            k = np.argmax(D_r)
            
            I.append(k)
            D_I[j] = D[k]
            
            # gamma, res, rank, s = LA.lstsq(D_I[:j+1].T, y, lapack_driver='gelsd')
            # gamma, istop, itn, res = lsmr(D_I[:j+1].T, y)[:4]
            gamma = np.linalg.solve(D_I[:j+1] @ D_I[:j+1].T, D_I[:j+1] @ y)
            if debug:
                print(gamma.shape)
                print(D_I[:j+1].T.shape)
                print(y.shape)
            r = y - D_I[:(j+1)].T @ gamma 
            if debug:
                print(np.sum(r * r))
                # print(res)

        X[i, I] = gamma
        
    return X

# Scipy lu_solve

In [4]:
def OMP_3(Y, T_0, D, rng=42, debug=False):
    # loss = np.empty(num_iter)
    # rng = np.random.default_rng(rng)

    X = np.zeros((Y.shape[0], D.shape[0]))

    for i, y in enumerate(Y):
        I = []
        D_I = np.zeros((T_0, D.shape[1]))
        r = y
        gamma = 0

        for j in range(T_0):
            D_r = np.abs(D @ r)
            k = np.argmax(D_r)
            
            I.append(k)
            D_I[j] = D[k]
            
            # gamma, res, rank, s = LA.lstsq(D_I[:j+1].T, y, lapack_driver='gelsd')
            # gamma, istop, itn, res = lsmr(D_I[:j+1].T, y)[:4]

            gram = D_I[:j+1] @ D_I[:j+1].T
            cov =  D_I[:j+1] @ y
            # gamma = LA.solve(gram, cov,
            #     overwrite_a = True,
            #     overwrite_b = True,
            #     assume_a = 'pos')
            gamma = LA.lu_solve((LA.lu_factor(gram, check_finite=False)), 
                                cov,
                                check_finite=False)
            
            if debug:
                print(gamma.shape)
                print(D_I[:j+1].T.shape)
                print(y.shape)
            r = y - gamma @ D_I[:(j+1)]  
            if debug:
                print(np.sum(r * r))
                # print(res)

        X[i, I] = gamma
        
    return X

## lu_solve with overwrite

In [5]:
def OMP_4(Y, T_0, D, rng=42, debug=False):
    # loss = np.empty(num_iter)
    # rng = np.random.default_rng(rng)

    X = np.zeros((Y.shape[0], D.shape[0]))

    for i, y in enumerate(Y):
        I = []
        D_I = np.zeros((T_0, D.shape[1]))
        r = y
        gamma = 0

        for j in range(T_0):
            D_r = np.abs(D @ r)
            k = np.argmax(D_r)
            
            I.append(k)
            D_I[j] = D[k]
            
            # gamma, res, rank, s = LA.lstsq(D_I[:j+1].T, y, lapack_driver='gelsd')
            # gamma, istop, itn, res = lsmr(D_I[:j+1].T, y)[:4]

            gram = D_I[:j+1] @ D_I[:j+1].T
            cov =  D_I[:j+1] @ y
            # gamma = LA.solve(gram, cov,
            #     overwrite_a = True,
            #     overwrite_b = True,
            #     assume_a = 'pos')
            # gamma = LA.lu_solve((LA.lu_factor(gram)), cov)
            gamma = LA.lu_solve((LA.lu_factor(gram, overwrite_a=True, check_finite=False)), 
                                 cov, overwrite_b=True, check_finite=False)
            
            if debug:
                print(gamma.shape)
                print(D_I[:j+1].T.shape)
                print(y.shape)
            r = y - gamma @ D_I[:(j+1)]  
            if debug:
                print(np.sum(r * r))
                # print(res)

        X[i, I] = gamma
        
    return X

# Scipy cho_solve

In [6]:
def OMP_5(Y, T_0, D, rng=42, debug=False):
    # loss = np.empty(num_iter)
    # rng = np.random.default_rng(rng)

    X = np.zeros((Y.shape[0], D.shape[0]))

    for i, y in enumerate(Y):
        I = []
        D_I = np.zeros((T_0, D.shape[1]))
        r = y
        gamma = 0

        for j in range(T_0):
            D_r = np.abs(D @ r)
            k = np.argmax(D_r)
            
            I.append(k)
            D_I[j] = D[k]
            
            # gamma, res, rank, s = LA.lstsq(D_I[:j+1].T, y, lapack_driver='gelsd')
            # gamma, istop, itn, res = lsmr(D_I[:j+1].T, y)[:4]

            gram = D_I[:j+1] @ D_I[:j+1].T
            cov =  D_I[:j+1] @ y
            # gamma = LA.solve(gram, cov,
            #     overwrite_a = True,
            #     overwrite_b = True,
            #     assume_a = 'pos')
            # gamma = LA.lu_solve((LA.lu_factor(gram)), cov)
            gamma = LA.cho_solve((LA.cho_factor(gram, overwrite_a=True, check_finite=False)), 
                                 cov, overwrite_b=True, check_finite=False)
            
            if debug:
                print(gamma.shape)
                print(D_I[:j+1].T.shape)
                print(y.shape)
            r = y - gamma @ D_I[:(j+1)]  
            if debug:
                print(np.sum(r * r))
                # print(res)

        X[i, I] = gamma
        
    return X

In [7]:
def OMP_verif(code):
    print(code)
    I = np.nonzero(code)
    print(I)
    print(code[I])

In [8]:
X, y = make_regression(n_samples = 50, n_features = 20, n_targets = 2, noise=4, random_state=0)

In [9]:
X.shape

(50, 20)

In [10]:
y.shape

(50, 2)

In [11]:
b_numpy = OMP(y.T, 1, X.T, debug=True)

(1,)
(50, 1)
(50,)
2449425.0712439856
[2449425.07124399]
(1,)
(50, 1)
(50,)
1817892.1699581572
[1817892.16995816]


In [12]:
OMP_verif(b_numpy)

[[  0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.          99.70035543   0.           0.           0.
    0.           0.           0.           0.           0.        ]
 [  0.         146.54061532   0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.        ]]
(array([0, 1]), array([11,  1]))
[ 99.70035543 146.54061532]


In [13]:
b_scipy = OMP_2(y.T, 1, X.T, debug=True)

(1,)
(50, 1)
(50,)
2449425.071243986
(1,)
(50, 1)
(50,)
1817892.1699581572


In [14]:
OMP_verif(b_scipy)

[[  0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.          99.70035543   0.           0.           0.
    0.           0.           0.           0.           0.        ]
 [  0.         146.54061532   0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.        ]]
(array([0, 1]), array([11,  1]))
[ 99.70035543 146.54061532]


In [15]:
b_scipy2 = OMP_3(y.T, 1, X.T, debug=True)

(1,)
(50, 1)
(50,)
2449425.071243986
(1,)
(50, 1)
(50,)
1817892.1699581572


In [16]:
OMP_verif(b_scipy2)

[[  0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.          99.70035543   0.           0.           0.
    0.           0.           0.           0.           0.        ]
 [  0.         146.54061532   0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.        ]]
(array([0, 1]), array([11,  1]))
[ 99.70035543 146.54061532]


In [17]:
b_scipy3 = OMP_4(y.T, 1, X.T, debug=True)

(1,)
(50, 1)
(50,)
2449425.071243986
(1,)
(50, 1)
(50,)
1817892.1699581572


In [18]:
OMP_verif(b_scipy3)

[[  0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.          99.70035543   0.           0.           0.
    0.           0.           0.           0.           0.        ]
 [  0.         146.54061532   0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.        ]]
(array([0, 1]), array([11,  1]))
[ 99.70035543 146.54061532]


In [19]:
b_scipy4 = OMP_5(y.T, 1, X.T, debug=True)

(1,)
(50, 1)
(50,)
2449425.071243986
(1,)
(50, 1)
(50,)
1817892.1699581572


In [20]:
OMP_verif(b_scipy4)

[[  0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.          99.70035543   0.           0.           0.
    0.           0.           0.           0.           0.        ]
 [  0.         146.54061532   0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.
    0.           0.           0.           0.           0.        ]]
(array([0, 1]), array([11,  1]))
[ 99.70035543 146.54061532]


In [21]:
np.allclose(b_scipy[np.nonzero(b_scipy)], b_numpy[np.nonzero(b_numpy)])

True

In [22]:
np.allclose(b_scipy2[np.nonzero(b_scipy2)], b_numpy[np.nonzero(b_numpy)])

True

In [23]:
np.allclose(b_scipy3[np.nonzero(b_scipy3)], b_numpy[np.nonzero(b_numpy)])

True

In [24]:
np.allclose(b_scipy4[np.nonzero(b_scipy4)], b_numpy[np.nonzero(b_numpy)])

True

## IT WORKS !!!

In [25]:
X, y = make_regression(n_samples = 50, n_features = 300, n_targets = 20_000, noise=4, random_state=0)

In [26]:
%timeit OMP(y.T, 1, X.T)

1.06 s ± 47.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
%timeit OMP_2(y.T, 1, X.T)

825 ms ± 16.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
%timeit OMP_3(y.T, 1, X.T)

1.19 s ± 38.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
%timeit OMP_4(y.T, 1, X.T)

1.13 s ± 13.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [30]:
%timeit OMP_5(y.T, 1, X.T)

1.15 s ± 14.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
%load_ext line_profiler
from line_profiler import profile

In [32]:
np_OMP = profile(OMP)
%lprun -f np_OMP np_OMP(y.T, 1, X.T)

Timer unit: 1e-07 s

Total time: 1.72226 s
File: C:\Users\richa\AppData\Local\Temp\ipykernel_21444\1184636726.py
Function: OMP at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def OMP(Y, T_0, D, rng=42, debug=False):
     2                                               # loss = np.empty(num_iter)
     3                                               # rng = np.random.default_rng(rng)
     4                                           
     5         1        284.0    284.0      0.0      X = np.zeros((Y.shape[0], D.shape[0]))
     6                                           
     7                                               # gram = D @ D.T
     8                                               # cov = D @ Y.T
     9                                               # X = sparse_encode(Y, 
    10                                               #                   D, 
    11                                               # 

In [33]:
sci_OMP = profile(OMP_2)
%lprun -f sci_OMP sci_OMP(y.T, 1, X.T)

Timer unit: 1e-07 s

Total time: 1.35903 s
File: C:\Users\richa\AppData\Local\Temp\ipykernel_21444\753983279.py
Function: OMP_2 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def OMP_2(Y, T_0, D, rng=42, debug=False):
     2                                               # loss = np.empty(num_iter)
     3                                               # rng = np.random.default_rng(rng)
     4                                           
     5         1        362.0    362.0      0.0      X = np.zeros((Y.shape[0], D.shape[0]))
     6                                           
     7                                               # gram = D @ D.T
     8                                               # cov = D @ Y.T
     9                                               # X = sparse_encode(Y, 
    10                                               #                   D, 
    11                                              

In [34]:
sci_OMP2 = profile(OMP_3)
%lprun -f sci_OMP2 sci_OMP2(y.T, 1, X.T)

Timer unit: 1e-07 s

Total time: 2.20889 s
File: C:\Users\richa\AppData\Local\Temp\ipykernel_21444\4250137686.py
Function: OMP_3 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def OMP_3(Y, T_0, D, rng=42, debug=False):
     2                                               # loss = np.empty(num_iter)
     3                                               # rng = np.random.default_rng(rng)
     4                                           
     5         1        425.0    425.0      0.0      X = np.zeros((Y.shape[0], D.shape[0]))
     6                                           
     7     20001     163779.0      8.2      0.7      for i, y in enumerate(Y):
     8     20000      53914.0      2.7      0.2          I = []
     9     20000     357583.0     17.9      1.6          D_I = np.zeros((T_0, D.shape[1]))
    10     20000      42459.0      2.1      0.2          r = y
    11     20000      41233.0      2.1      0.2

In [35]:
sci_OMP3 = profile(OMP_4)
%lprun -f sci_OMP3 sci_OMP3(y.T, 1, X.T)

Timer unit: 1e-07 s

Total time: 2.13472 s
File: C:\Users\richa\AppData\Local\Temp\ipykernel_21444\339722602.py
Function: OMP_4 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def OMP_4(Y, T_0, D, rng=42, debug=False):
     2                                               # loss = np.empty(num_iter)
     3                                               # rng = np.random.default_rng(rng)
     4                                           
     5         1        283.0    283.0      0.0      X = np.zeros((Y.shape[0], D.shape[0]))
     6                                           
     7     20001     167450.0      8.4      0.8      for i, y in enumerate(Y):
     8     20000      54599.0      2.7      0.3          I = []
     9     20000     346395.0     17.3      1.6          D_I = np.zeros((T_0, D.shape[1]))
    10     20000      40893.0      2.0      0.2          r = y
    11     20000      34618.0      1.7      0.2 

In [36]:
sci_OMP4 = profile(OMP_5)
%lprun -f sci_OMP4 sci_OMP4(y.T, 1, X.T)

Timer unit: 1e-07 s

Total time: 2.14211 s
File: C:\Users\richa\AppData\Local\Temp\ipykernel_21444\2536862899.py
Function: OMP_5 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def OMP_5(Y, T_0, D, rng=42, debug=False):
     2                                               # loss = np.empty(num_iter)
     3                                               # rng = np.random.default_rng(rng)
     4                                           
     5         1        246.0    246.0      0.0      X = np.zeros((Y.shape[0], D.shape[0]))
     6                                           
     7     20001     165204.0      8.3      0.8      for i, y in enumerate(Y):
     8     20000      55105.0      2.8      0.3          I = []
     9     20000     376280.0     18.8      1.8          D_I = np.zeros((T_0, D.shape[1]))
    10     20000      42897.0      2.1      0.2          r = y
    11     20000      35593.0      1.8      0.2